In [33]:
import pandas as pd
from datetime import timedelta




In [34]:
df = pd.read_csv('Changes.csv')

In [35]:
df['day'] = pd.to_datetime(df['day'])
df = df.sort_values('day')

In [36]:
release_dates = pd.to_datetime([
    "2024-11-16", "2024-10-12", "2024-09-10",
    "2024-02-20", "2024-01-19", "2023-07-07", "2022-10-01"
])

#release_dates = release_dates.strftime('%b %d, %Y')

release_dates

DatetimeIndex(['2024-11-16', '2024-10-12', '2024-09-10', '2024-02-20',
               '2024-01-19', '2023-07-07', '2022-10-01'],
              dtype='datetime64[ns]', freq=None)

In [41]:
def analyze_release_impact(df, release_date, intervals=60):
    before = df[(df['day'] >= release_date - timedelta(days=intervals)) & 
                (df['day'] < release_date)]
    
    after = df[(df['day'] > release_date) & 
               (df['day'] <= release_date + timedelta(days=intervals))]
    
    before_avg = before['Users'].mean()
    after_avg = after['Users'].mean()
    diff = after_avg - before_avg if pd.notnull(after_avg) and pd.notnull(before_avg) else None
    pct_change = (diff / before_avg * 100) if before_avg and diff is not None else None

    return {
        "Release_Date": release_date.strftime('%b %d, %Y'),
        "Before_Avg": round(before_avg, 2) if pd.notnull(before_avg) else None,
        "After_Avg": round(after_avg, 2) if pd.notnull(after_avg) else None,
        "Change": round(diff, 2) if diff is not None else None,
        "Percent_Change": round(pct_change, 2) if pct_change is not None else None,
        "Effect":   (
            "Positive" if diff and diff > 0 else
            "Negative" if diff and diff < 0 else
            "Neutral"
        )
    }


In [43]:
# Apply the analysis function to each release date
analysis_results = [analyze_release_impact(df, date) for date in release_dates]

# Convert to a DataFrame
impact_df = pd.DataFrame(analysis_results)

def highlight_effect(row):
    if row['Effect'] == 'Positive':
        return ['background-color: #d4edda'] * len(row)  # Light green
    #elif row['Effect'] == 'Negative':
        #return ['background-color: #f8d7da'] * len(row)  # Light red
    else:
        return [''] * len(row)

# Format and style the DataFrame
impact_df.style.apply(highlight_effect, axis=1).format({
    'Before_Avg': '{:,.2f}',
    'After_Avg': '{:,.2f}',
    'Change': '{:,.2f}',
    'Percent_Change': '{:+.2f}%'
})


# Display the result


,Release_Date,Before_Avg,After_Avg,Change,Percent_Change,Effect
0,"Nov 16, 2024",nan,nan,nan,+nan%,Neutral
1,"Oct 12, 2024",nan,nan,nan,+nan%,Neutral
2,"Sep 10, 2024",nan,nan,nan,+nan%,Neutral
3,"Feb 20, 2024",843.82,nan,nan,+nan%,Neutral
4,"Jan 19, 2024",880.63,nan,nan,+nan%,Neutral
5,"Jul 07, 2023",987.70,934.23,-53.47,-5.41%,Negative
6,"Oct 01, 2022",nan,nan,nan,+nan%,Neutral
